In [1]:
import pickle
import matplotlib.pyplot as plt
import torch
import numpy as np
from src.Dataset_loader import unpickle, CustomImageDataset

def show(img):
    img = img/255
    plt.imshow(np.transpose(img.numpy(), (1, 2, 0)), interpolation='nearest')
    plt.show()


In [2]:
CIFAR10_train_dataset = CustomImageDataset()
CIFAR10_test_dataset = CustomImageDataset()

In [3]:
CIFAR10_train_dataset.load_batch('data/cifar-10-batches-py/data_batch_1')
CIFAR10_train_dataset.load_batch('data/cifar-10-batches-py/data_batch_2')
CIFAR10_train_dataset.load_batch('data/cifar-10-batches-py/data_batch_3')
CIFAR10_train_dataset.load_batch('data/cifar-10-batches-py/data_batch_4')
CIFAR10_train_dataset.load_batch('data/cifar-10-batches-py/data_batch_5')

In [4]:
CIFAR10_test_dataset.load_batch('data/cifar-10-batches-py/test_batch')

In [5]:
train_loader = torch.utils.data.DataLoader(CIFAR10_train_dataset, batch_size=32,
                                         shuffle=False, num_workers=2)

In [6]:
test_loader = torch.utils.data.DataLoader(CIFAR10_test_dataset, batch_size=32,
                                         shuffle=False, num_workers=2)

In [61]:
from torch import nn, optim
import torch.nn.functional as F
